# Day 6 - Lab 1: Building RAG Systems

**Objective:** Build a RAG (Retrieval-Augmented Generation) system orchestrated by LangGraph, scaling in complexity from a simple retriever to a multi-agent team that includes a grader and a router.

**Estimated Time:** 180 minutes

**Introduction:**
Welcome to Day 6! Today, we build one of the most powerful and common patterns for enterprise AI: a system that can answer questions about your private documents. We will use LangGraph to create a 'research team' of AI agents. Each agent will have a specific job, and LangGraph will act as the manager, orchestrating their collaboration to find the best possible answer.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We need several libraries for this lab. `langgraph` is the core orchestrator, `langchain` provides the building blocks, `faiss-cpu` is for our vector store, and `pypdf` is for loading documents.

**Model Selection:**
For RAG and agentic workflows, models with strong instruction-following and reasoning are best. `gpt-4.1`, `o3`, or `gemini-2.5-pro` are excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `load_artifact()`: To read the project documents that will form our knowledge base.

In [44]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('langgraph')
install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('faiss-cpu')
install_if_missing('pypdf')

from utils import setup_llm_client, load_artifact
from typing import List, TypedDict
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, END

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")
llm = ChatOpenAI(model=model_name)
embeddings = OpenAIEmbeddings()

faiss-cpu not found, installing...


2025-09-29 16:36:24,843 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: Building the Knowledge Base

An agent is only as smart as the information it can access. We will create a vector store containing all the project artifacts we've created so far. This will be our agent's 'knowledge base'.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def create_knowledge_base(file_paths):
    """Loads documents from given paths and creates a FAISS vector store.""" 
    all_docs = []
    for path in file_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            docs = loader.load()
            for doc in docs:
                doc.metadata={"source": path} # Add source metadata
            all_docs.extend(docs)
        else:
            print(f"Warning: Artifact not found at {full_path}")

    if not all_docs:
        print("No documents found to create knowledge base.")
        return None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(all_docs)
    
    print(f"Creating vector store from {len(splits)} document splits...")
    vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore.as_retriever()

all_artifact_paths = ["artifacts/day1_prd.md", "artifacts/schema.sql", "artifacts/adr_001_database_choice.md"]
retriever = create_knowledge_base(all_artifact_paths)

Creating vector store from 36 document splits...


## Step 3: The Challenges

### Challenge 1 (Foundational): A Simple RAG Graph

**Task:** Build a simple LangGraph with two nodes: one to retrieve documents and one to generate an answer.

> **Tip:** Think of `AgentState` as the shared 'whiteboard' for your agent team. Every agent (or 'node' in the graph) can read from and write to this state, allowing them to pass information to each other as they work on a problem.

**Instructions:**
1.  Define the state for your graph using a `TypedDict`. It should contain keys for `question` and `documents`.
2.  Create a "Retriever" node. This is a Python function that takes the state, uses the `retriever` to get relevant documents, and updates the state with the results.
3.  Create a "Generator" node. This function takes the state, creates a prompt with the question and retrieved documents, calls the LLM, and stores the answer.
4.  Build the `StateGraph`, add the nodes, and define the edges (`RETRIEVE` -> `GENERATE`).
5.  Compile the graph and invoke it with a question about your project.

**Expected Quality:** A functional graph that can answer a simple question (e.g., "What is the purpose of this project?") by retrieving context from the project artifacts.

In [ ]:
class SimpleAgentState(TypedDict):
    question: str
    documents: List[Document]
    answer: str

def retrieve(state):
    print("---NODE: RETRIEVE DOCUMENTS---")
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def generate(state):
    print("---NODE: GENERATE ANSWER---")
    question = state["question"]
    documents = state["documents"]
    prompt = f"""You are an assistant for question-answering tasks. Use the following retrieved context to answer the question. If you don't know the answer, just say that you don't know.\n\nQuestion: {question}\n\nContext: {documents}\n\nAnswer:"""
    answer = llm.invoke(prompt).content
    return {"answer": answer}

workflow_v1 = StateGraph(SimpleAgentState)
workflow_v1.add_node("RETRIEVE", retrieve)
workflow_v1.add_node("GENERATE", generate)
workflow_v1.set_entry_point("RETRIEVE")
workflow_v1.add_edge("RETRIEVE", "GENERATE")
workflow_v1.add_edge("GENERATE", END)

app_v1 = workflow_v1.compile()

print("\n--- Invoking Simple RAG Graph ---")
inputs = {"question": "What is the purpose of this project according to the PRD?"}
result = app_v1.invoke(inputs)
print(f"Final Answer: {result['answer']}")

---NODE: RETRIEVE DOCUMENTS---
---NODE: GENERATE ANSWER---
Final Answer: According to the Product Requirements Document (PRD), the purpose of this project is to streamline and enhance the onboarding experience for new hires by providing an integrated, user-friendly environment where they can complete tasks, learn about company culture, and interact with mentors and peers. The goal is to create an efficient onboarding process that reduces the time to productivity for new hires while increasing their engagement and satisfaction.


### Challenge 2 (Intermediate): A Graph with a Grader Agent

**Task:** Add a second agent to your graph that acts as a "Grader," deciding if the retrieved documents are relevant enough to answer the question.

> **What is a conditional edge?** It's a decision point. After a node completes its task (like our 'Grader'), the conditional edge runs a function to decide which node to go to next. This allows your agent to change its plan based on new information.

**Instructions:**
1.  Keep your `RETRIEVE` and `GENERATE` nodes from the previous challenge.
2.  Create a new "Grader" node. This function takes the state (question and documents) and calls an LLM with a specific prompt: "Based on the question and the following documents, is the information sufficient to answer the question? Answer with only 'yes' or 'no'."
3.  Add a **conditional edge** to your graph. After the `RETRIEVE` node, the graph should go to the `GRADE` node. After the `GRADE` node, it should check the grader's response. If 'yes', it proceeds to the `GENERATE` node. If 'no', it goes to an `END` node, concluding that it cannot answer the question.

**Expected Quality:** A more robust graph that can gracefully handle cases where its knowledge base doesn't contain the answer, preventing it from hallucinating.

In [43]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_core.documents import Document

# Challenge 2: RAG System with Grader Agent
class GraderAgentState(TypedDict):
    question: str
    documents: List[Document]
    answer: str
    grade: str

def retriever_node(state: GraderAgentState) -> GraderAgentState:
    print("---NODE: RETRIEVE DOCUMENTS---")
    question = state["question"]
    docs = retriever.get_relevant_documents(question)
    return {
        "question": question,
        "documents": docs,
        "answer": "",
        "grade": ""
    }

def grade_documents(state: GraderAgentState) -> GraderAgentState:
    print("---NODE: GRADE DOCUMENTS---")
    question = state["question"]
    docs = state["documents"]
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = (
        "You are a grader assessing relevance of retrieved documents to a user question. "
        "If the documents contain keywords related to the user question, grade as relevant. "
        "Grade 'yes' or 'no'.\n\nRetrieved Documents: " + context + "\n\nUser Question: " + question
    )
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful grader. Only answer 'yes' or 'no'."},
            {"role": "user", "content": prompt}
        ]
    )
    grade = response.choices[0].message.content.strip()
    return {**state, "grade": grade}

def decide_to_generate(state: GraderAgentState):
    print("---NODE: CONDITIONAL EDGE---")
    if state["grade"].lower() == "yes":
        print("DECISION: Documents are relevant. Proceed to generation.")
        return "GENERATE"
    else:
        print("DECISION: Documents are not relevant. End process.")
        return END

def generator_node(state: GraderAgentState) -> GraderAgentState:
    print("---NODE: GENERATE ANSWER---")
    question = state["question"]
    docs = state["documents"]
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = (
        "You are an assistant for question-answering tasks. Use the following retrieved context to answer the question. If you don't know the answer, just say that you don't know.\n\n"
        f"Question: {question}\n\nContext: {context}\n\nAnswer:"
    )
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    answer = response.choices[0].message.content.strip()
    return {**state, "answer": answer}

graph2 = StateGraph(GraderAgentState)
graph2.add_node("RETRIEVE", retriever_node)
graph2.add_node("GRADE", grade_documents)
graph2.add_node("GENERATE", generator_node)
graph2.set_entry_point("RETRIEVE")
graph2.add_edge("RETRIEVE", "GRADE")
graph2.add_conditional_edges("GRADE", decide_to_generate)
graph2.add_edge("GENERATE", END)

app2 = graph2.compile()

print("\n--- Invoking Grader Graph with a relevant question ---")
inputs = {"question": "What database schema will we use?", "documents": [], "answer": "", "grade": ""}
result = app2.invoke(inputs)
print(f"Final Answer: {result.get('answer', 'Could not answer question.')}")

print("\n--- Invoking Grader Graph with an irrelevant question ---")
inputs = {"question": "What is the weather in Paris?", "documents": [], "answer": "", "grade": ""}
result = app2.invoke(inputs)
print(f"Final Answer: {result.get('answer', 'Could not answer question.')}")


--- Invoking Grader Graph with a relevant question ---
---NODE: RETRIEVE DOCUMENTS---
---NODE: GRADE DOCUMENTS---
---NODE: GRADE DOCUMENTS---
---NODE: CONDITIONAL EDGE---
DECISION: Documents are relevant. Proceed to generation.
---NODE: GENERATE ANSWER---
---NODE: CONDITIONAL EDGE---
DECISION: Documents are relevant. Proceed to generation.
---NODE: GENERATE ANSWER---
Final Answer: We will use PostgreSQL with the pgvector extension as the database schema. This unified database will store both the relational metadata (such as text content, author, permissions) and the vector embeddings needed for semantic search in the new hire onboarding tool.

--- Invoking Grader Graph with an irrelevant question ---
---NODE: RETRIEVE DOCUMENTS---
Final Answer: We will use PostgreSQL with the pgvector extension as the database schema. This unified database will store both the relational metadata (such as text content, author, permissions) and the vector embeddings needed for semantic search in the new

### Challenge 3 (Advanced): A Multi-Agent Research Team

**Task:** Build a sophisticated "research team" of specialized agents that includes a router to delegate tasks to the correct specialist.

**Instructions:**
1.  **Specialize your retriever:** Create two separate retrievers. One for the PRD (`prd_retriever`) and one for the technical documents (`tech_retriever` for schema and ADRs).
2.  **Define the Agents:**
    * `ProjectManagerAgent`: This will be the entry point and will act as a router. It uses an LLM to decide whether the user's question is about product requirements or technical details, and routes to the appropriate researcher.
    * `PRDResearcherAgent`: A node that uses the `prd_retriever`.
    * `TechResearcherAgent`: A node that uses the `tech_retriever`.
    * `SynthesizerAgent`: A node that takes the collected documents from either researcher and synthesizes a final answer.
3.  **Build the Graph:** Use conditional edges to orchestrate the flow: The entry point is the `ProjectManager`, which then routes to either the `PRD_RESEARCHER` or `TECH_RESEARCHER`. Both of those nodes should then route to the `SYNTHESIZE` node, which then goes to the `END`.

**Expected Quality:** A highly advanced agentic system that mimics a real-world research workflow, including a router and specialist roles, to improve the accuracy and efficiency of the RAG process.

In [ ]:
# Challenge 3: Multi-Agent Research Team with Router and Specialists
from typing import TypedDict, List
from langgraph.graph import StateGraph
from langchain_core.documents import Document
import os

# 1. Specialized retrievers
prd_artifact_paths = ["artifacts/day1_prd.md"]
tech_artifact_paths = ["artifacts/schema.sql", "artifacts/adr_001_database_choice.md"]
prd_retriever = create_knowledge_base(prd_artifact_paths)
tech_retriever = create_knowledge_base(tech_artifact_paths)

# 2. Define the state for the graph
class TeamState(TypedDict):
    question: str
    prd_documents: List[Document]
    tech_documents: List[Document]
    synthesized_answer: str
    route_decision: str

# 3. ProjectManagerAgent (router)
def project_manager_node(state: TeamState) -> TeamState:
    question = state["question"]
    prompt = (
        "You are a project manager. "
        "Decide if the user's question is about product requirements (PRD) or technical details (schema, ADRs). "
        "Answer with only 'prd' or 'tech'.\nQuestion: " + question
    )
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a project manager. Only answer 'prd' or 'tech'."},
            {"role": "user", "content": prompt}
        ]
    )
    route_decision = response.choices[0].message.content.strip().lower()
    return {**state, "route_decision": route_decision}

# 4. PRDResearcherAgent
def prd_researcher_node(state: TeamState) -> TeamState:
    question = state["question"]
    docs = prd_retriever.get_relevant_documents(question)
    return {**state, "prd_documents": docs}

# 5. TechResearcherAgent
def tech_researcher_node(state: TeamState) -> TeamState:
    question = state["question"]
    docs = tech_retriever.get_relevant_documents(question)
    return {**state, "tech_documents": docs}

# 6. SynthesizerAgent
def synthesizer_node(state: TeamState) -> TeamState:
    question = state["question"]
    prd_docs = state.get("prd_documents", [])
    tech_docs = state.get("tech_documents", [])
    context = "\n\n".join([doc.page_content for doc in prd_docs + tech_docs])
    prompt = f"Synthesize an answer to the following question using the provided context.\nQuestion: {question}\nContext: {context}"
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful synthesizer."},
            {"role": "user", "content": prompt}
        ]
    )
    synthesized_answer = response.choices[0].message.content
    return {**state, "synthesized_answer": synthesized_answer}

# 7. Conditional router function
def route_decision_fn(state: TeamState) -> str:
    decision = state.get("route_decision", "tech")
    if "prd" in decision:
        return "PRD_RESEARCHER"
    else:
        return "TECH_RESEARCHER"

# 8. Build the graph
team_graph = StateGraph(TeamState)
team_graph.add_node("PROJECT_MANAGER", project_manager_node)
team_graph.add_node("PRD_RESEARCHER", prd_researcher_node)
team_graph.add_node("TECH_RESEARCHER", tech_researcher_node)
team_graph.add_node("SYNTHESIZER", synthesizer_node)
team_graph.add_node("END", lambda state: state)
team_graph.add_conditional_edges("PROJECT_MANAGER", route_decision_fn, {
    "PRD_RESEARCHER": "PRD_RESEARCHER",
    "TECH_RESEARCHER": "TECH_RESEARCHER"
})
team_graph.add_edge("PRD_RESEARCHER", "SYNTHESIZER")
team_graph.add_edge("TECH_RESEARCHER", "SYNTHESIZER")
team_graph.add_edge("SYNTHESIZER", "END")
team_graph.set_entry_point("PROJECT_MANAGER")

# 9. Compile and run the graph
team_app = team_graph.compile()

# Example question (try both PRD and tech questions)
question = "What are the main product requirements?"  # Try changing to a technical question
state = {
    "question": question,
    "prd_documents": [],
    "tech_documents": [],
    "synthesized_answer": "",
    "route_decision": ""
}
result = team_app.invoke(state)
print("Synthesized Answer:", result["synthesized_answer"])

# Print a clear final answer summary
print(f"Final Answer: {result['synthesized_answer']}")

Creating vector store from 14 document splits...
Creating vector store from 22 document splits...
Creating vector store from 22 document splits...
Synthesized Answer: The main product requirements for the Onboarding Platform are as follows:

**1. Core Features:**
- **Onboarding Checklist:** A comprehensive checklist that enables new hires to view and complete onboarding tasks and goals efficiently.
- **Interactive Modules:** Tools or modules that provide engaging onboarding content, likely including company culture, processes, and introductions.
- **Customizable User Profiles:** Allow new hires to personalize their profiles and track their onboarding progress.

**2. Integration & Access:**
- Integration with existing company calendar systems for scheduling and task reminders.
- Access to current HR content and resources for use in onboarding modules.

**3. Usability & Engagement:**
- The platform must be user-friendly and designed to increase new hire engagement and satisfaction.
- New

## Lab Conclusion

Incredible work! You have now built a truly sophisticated AI system. You've learned how to create a knowledge base for an agent and how to use LangGraph to orchestrate a team of specialized agents to solve a complex problem. You progressed from a simple RAG chain to a system that includes quality checks (the Grader) and intelligent task delegation (the Router). These are the core patterns for building production-ready RAG applications.

> **Key Takeaway:** LangGraph allows you to define complex, stateful, multi-agent workflows as a graph. Using nodes for agents and conditional edges for decision-making enables the creation of sophisticated systems that can reason, delegate, and collaborate to solve problems more effectively than a single agent could alone.